Download data from [CryptoWatch API](https://cryptowat.ch/docs/api) for educational purposes.

In [167]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

### Markets

In [3]:
resp = requests.get('https://api.cryptowat.ch/markets')

In [4]:
resp.ok

True

In [6]:
resp.json()['result'][:2]

[{'id': 1,
  'exchange': 'bitfinex',
  'pair': 'btcusd',
  'active': True,
  'route': 'https://api.cryptowat.ch/markets/bitfinex/btcusd'},
 {'id': 2,
  'exchange': 'bitfinex',
  'pair': 'ltcusd',
  'active': True,
  'route': 'https://api.cryptowat.ch/markets/bitfinex/ltcusd'}]

In [9]:
markets = pd.DataFrame.from_records(resp.json()['result'], index='id')
markets.head()

,exchange,pair,active,route
id,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd
3,bitfinex,ltcbtc,True,https://api.cryptowat.ch/markets/bitfinex/ltcbtc
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd
5,bitfinex,ethbtc,True,https://api.cryptowat.ch/markets/bitfinex/ethbtc


### Exchanges

In [15]:
VALID_EXCHANGES = [x.strip() for x in """
bitfinex
coinbase-pro
bitstamp
kraken
cexio
okcoin
bitmex
mexbt
huobi
poloniex
bittrex
okex
hitbtc
""".split('\n') if x.strip()]
VALID_EXCHANGES

['bitfinex',
 'coinbase-pro',
 'bitstamp',
 'kraken',
 'cexio',
 'okcoin',
 'bitmex',
 'mexbt',
 'huobi',
 'poloniex',
 'bittrex',
 'okex',
 'hitbtc']

### Cryptos

In [53]:
VALID_CRYPTOS = ['BTC', 'LTC', 'ETH']

In [60]:
CRYPTOS_REGEX = '|'.join(["^{}usdt*$".format(c.lower()) for c in VALID_CRYPTOS])
CRYPTOS_REGEX

'^btcusdt*$|^ltcusdt*$|^ethusdt*$'

### Pairs to download

In [61]:
markets.head()

,exchange,pair,active,route,Should Download?
id,,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd,True
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd,True
3,bitfinex,ltcbtc,True,https://api.cryptowat.ch/markets/bitfinex/ltcbtc,False
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd,True
5,bitfinex,ethbtc,True,https://api.cryptowat.ch/markets/bitfinex/ethbtc,False


In [62]:
markets['Should Download?'] = markets['pair'].str.contains(CRYPTOS_REGEX) & markets['exchange'].isin(VALID_EXCHANGES)

In [67]:
markets['symbol'] = markets['pair'].str[:3].str.upper()

In [89]:
# markets['OHLC URL'] = markets.apply(lambda row: URL_TEMPLATE.format(exchange=row['exchange'], pair=row['pair']), axis=1)

In [90]:
to_download = markets.loc[markets['Should Download?']]

In [91]:
to_download.head()

,exchange,pair,active,route,Should Download?,symbol,OHLC URL
id,,,,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd,True,BTC,https://api.cryptowat.ch/markets/bitfinex/btcu...
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd,True,LTC,https://api.cryptowat.ch/markets/bitfinex/ltcu...
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd,True,ETH,https://api.cryptowat.ch/markets/bitfinex/ethu...
65,coinbase-pro,btcusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,BTC,https://api.cryptowat.ch/markets/coinbase-pro/...
68,coinbase-pro,ethusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,ETH,https://api.cryptowat.ch/markets/coinbase-pro/...


### OHLC

In [122]:
url = 'https://api.cryptowat.ch/markets/bitfinex/ltcusdt/ohlc'

In [129]:
resp = requests.get(url, params={'periods': 86400})

In [130]:
resp.ok

True

In [125]:
resp.json()['result']['86400'][:2]

[[1555113600, 77.3, 79.139, 77.3, 78.784, 4.51065, 353.00803185],
 [1555200000, 79.657, 80.909, 76.88, 77.439, 224.80873319, 17956.80318777912]]

From:

In [132]:
pd.to_datetime(resp.json()['result']['86400'][-1][0], unit='s')

Timestamp('2019-07-22 00:00:00')

Up to:

In [131]:
pd.to_datetime(resp.json()['result']['86400'][0][0], unit='s')

Timestamp('2019-04-13 00:00:00')

In [118]:
ohlc = pd.DataFrame.from_records(
    resp.json()['result']['86400'],
    columns=['CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'VolumeUSD']
)

In [120]:
ohlc['DateTime'] = pd.to_datetime(ohlc['CloseTime'], unit='s')

In [121]:
ohlc.head()

,CloseTime,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,VolumeUSD,DateTime
0,1555113600,77.300,79.139,77.300,78.784,4.510650,353.008032,2019-04-13
1,1555200000,79.657,80.909,76.880,77.439,224.808733,17956.803188,2019-04-14
2,1555286400,77.400,82.320,76.725,82.320,211.321604,16416.498960,2019-04-15
3,1555372800,81.183,81.505,74.500,78.892,227.123199,17550.141392,2019-04-16
4,1555459200,78.204,80.680,78.124,80.367,80.081169,6280.867545,2019-04-17


In [109]:
ohlc.shape

(101, 8)

### Download 

In [135]:
to_download.head()

,exchange,pair,active,route,Should Download?,symbol,OHLC URL
id,,,,,,,
1,bitfinex,btcusd,True,https://api.cryptowat.ch/markets/bitfinex/btcusd,True,BTC,https://api.cryptowat.ch/markets/bitfinex/btcu...
2,bitfinex,ltcusd,True,https://api.cryptowat.ch/markets/bitfinex/ltcusd,True,LTC,https://api.cryptowat.ch/markets/bitfinex/ltcu...
4,bitfinex,ethusd,True,https://api.cryptowat.ch/markets/bitfinex/ethusd,True,ETH,https://api.cryptowat.ch/markets/bitfinex/ethu...
65,coinbase-pro,btcusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,BTC,https://api.cryptowat.ch/markets/coinbase-pro/...
68,coinbase-pro,ethusd,True,https://api.cryptowat.ch/markets/coinbase-pro/...,True,ETH,https://api.cryptowat.ch/markets/coinbase-pro/...


In [149]:
URL_TEMPLATE = 'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc'
URL_TEMPLATE

'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc'

In [170]:
def download_ohlc_into_csv(exchange, pair, symbol, destination='crypto_data'):
    url = URL_TEMPLATE.format(exchange=exchange, pair=pair)
    
    identifier = '{exchange}_{symbol}'.format(exchange=exchange, symbol=symbol.lower())

    resp = requests.get(url, params={'periods': 86400})
    if not resp.ok:
        return (identifier, False)

    resp.json()['result']['86400'][:2]

    df = pd.DataFrame.from_records(
        resp.json()['result']['86400'],
        columns=['CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'VolumeUSD']
    )

    df['DateTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    file_name = "{}/{}.csv".format(destination, identifier)
    df.to_csv(file_name, index=False)
    return (identifier, True)

Just a quick test:

In [165]:
download_ohlc_into_csv('coinbase-pro', 'ethusd', 'ETH')

('coinbase-pro_eth', True)

In [166]:
!head -n 3 crypto_data/coinbase-pro_eth.csv

CloseTime,OpenPrice,HighPrice,LowPrice,ClosePrice,Volume,VolumeUSD,DateTime
1521072000,690.32,702.5,585.52,614.63,152695.72,96964136.0,2018-03-15
1521158400,614.63,621.47,571.16,610.99,148955.0,89051080.0,2018-03-16


In [172]:
with ThreadPoolExecutor(max_workers=10) as ex:
    for exchange, pair, symbol in to_download[['exchange', 'pair', 'symbol']].values:
        futures = [ex.submit(download_ohlc_into_csv, exchange, pair, symbol)]
        for future in as_completed(futures):
            print(future.result())

('bitfinex_btc', True)
('bitfinex_ltc', True)
('bitfinex_eth', True)
('coinbase-pro_btc', True)
('coinbase-pro_eth', True)
('coinbase-pro_ltc', True)
('bitstamp_btc', True)
('bitstamp_ltc', True)
('bitstamp_eth', True)
('kraken_btc', True)
('kraken_ltc', True)
('kraken_eth', True)
('cexio_btc', True)
('cexio_ltc', True)
('cexio_eth', True)
('okcoin_btc', True)
('okcoin_ltc', True)
('mexbt_btc', True)
('poloniex_eth', True)
('poloniex_btc', True)
('poloniex_ltc', True)
('bittrex_btc', True)
('bittrex_eth', True)
('bittrex_ltc', True)
('okex_btc', True)
('okex_ltc', True)
('okex_eth', True)
('hitbtc_ltc', True)
('hitbtc_btc', True)
('bittrex_btc', True)
('hitbtc_eth', True)
('bittrex_eth', True)
('huobi_btc', True)
('huobi_ltc', True)
('huobi_eth', True)
('bittrex_ltc', True)
('bitfinex_btc', True)
('bitfinex_eth', True)
('bitfinex_ltc', True)
